In [6]:
from tensorflow.keras.datasets import mnist

(train_inputs, train_labels), (test_inputs, test_labels) = mnist.load_data()

print(train_inputs.shape, train_labels.shape, test_inputs.shape, test_labels.shape)  # 얘네는 numpy array임

print(test_inputs.type)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [5]:
train_labels

,0
0,0
1,0
2,0
3,0
4,0
...,...
227295,0
227296,0
227297,0
227298,0


In [9]:
import pandas as pd
import numpy as np

train_inputs, train_labels = np.array(pd.read_csv('./cover_train.csv')), np.array(pd.read_csv('./cover_train_label.csv'))
test_inputs, test_labels = np.array(pd.read_csv('./cover_test.csv')), np.array(pd.read_csv('./cover_test_label.csv'))

train_labels, test_labels = train_labels.reshape(-1,), test_labels.reshape(-1,)

print(train_inputs.shape, train_labels.shape, test_inputs.shape, test_labels.shape)

(227300, 10) (227300,) (58746, 10) (58746,)


In [21]:
import pandas as pd
import numpy as np
x = pd.read_csv('./통_x.csv')
y = pd.read_csv('./통_y.csv')
anomal_index = y.index[y['0']==1]  # 비정상
nomal_index = y.index[y['0']==0]   # 정상
normal_y = y.iloc[nomal_index]
anomal_y = y.iloc[anomal_index]
normal_x = x.iloc[nomal_index]
anomal_x = x.iloc[anomal_index]
#normal_y
#anomal_y
normal_y.to_csv('./normal_y.csv', index=False)
anomal_y.to_csv('./anomal_y.csv', index=False)
normal_x.to_csv('./normal_x.csv', index=False)
anomal_x.to_csv('./anomal_x.csv', index=False)

In [23]:
anomal_index

Int64Index([   824,    827,    830,    832,    833,    835,    837,    841,
               842,    844,
            ...
            174326, 174327, 174328, 174717, 174938, 175162, 176516, 176739,
            176981, 177192],
           dtype='int64', length=2747)

In [ ]:
x = pd.read_csv('./normal_x.csv')
y = pd.read_csv('./normal_y.csv')

to_test_x = x.iloc[:56000]
to_test_y = y.iloc[:56000]

real_x = x.iloc[56000:]
real_y = y.iloc[56000:]

to_test_x.to_csv('./to_test_x.csv', index=False)
to_test_y.to_csv('./to_test_y.csv', index=False)
real_x.to_csv('./real_x.csv', index=False)
real_y.to_csv('./real_y.csv', index=False)

In [14]:
train_digits=list([0])
train_digits

[0]

In [16]:
test_digits=list(range(2))
test_digits

[0, 1]

In [ ]:
def lrelu(x, leak=0.2, name='lrelu'):
	return tf.maximum(x, leak*x)

def build_dense(input_vector,unit_no,activation):    
    return tf.layers.dense(input_vector,unit_no,activation=activation,
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            bias_initializer=tf.zeros_initializer())

class MLP_VAE:
    def __init__(self,input_dim,lat_dim, outliers_fraction):
       # input_paras:
           # input_dim: input dimension for X
           # lat_dim: latent dimension for Z
           # outliers_fraction: pre-estimated fraction of outliers in trainning dataset
        
        self.outliers_fraction = outliers_fraction # for computing the threshold of anomaly score       
        self.input_dim = input_dim
        self.lat_dim = lat_dim # the lat_dim can exceed input_dim    
        
        self.input_X = tf.placeholder(tf.float32,shape=[None,self.input_dim],name='source_x')
        
        self.learning_rate = 0.0005
        self.batch_size =  32
        # batch_size should be smaller than normal setting for getting
        # a relatively lower anomaly-score-threshold
        self.train_iter = 3000
        self.hidden_units = 128
        self._build_VAE()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.pointer = 0
        
    def _encoder(self):
        with tf.variable_scope('encoder',reuse=tf.AUTO_REUSE):
            l1 = build_dense(self.input_X,self.hidden_units,activation=lrelu)
#            l1 = tf.nn.dropout(l1,0.8)
            l2 = build_dense(l1,self.hidden_units,activation=lrelu)
#            l2 = tf.nn.dropout(l2,0.8)          
            mu = tf.layers.dense(l2,self.lat_dim)
            sigma = tf.layers.dense(l2,self.lat_dim,activation=tf.nn.softplus)
            sole_z = mu + sigma *  tf.random_normal(tf.shape(mu),0,1,dtype=tf.float32)
        return mu,sigma,sole_z
        
    def _decoder(self,z):
        with tf.variable_scope('decoder',reuse=tf.AUTO_REUSE):
            l1 = build_dense(z,self.hidden_units,activation=lrelu)
#            l1 = tf.nn.dropout(l1,0.8)
            l2 = build_dense(l1,self.hidden_units,activation=lrelu)
#            l2 = tf.nn.dropout(l2,0.8)
            recons_X = tf.layers.dense(l2,self.input_dim)        # 다시 원래 차원으로 복원
        return recons_X           # 복원된 x

    def _build_VAE(self):
        self.mu_z,self.sigma_z,sole_z = self._encoder()
        self.recons_X = self._decoder(sole_z)
        
        with tf.variable_scope('loss'):
            KL_divergence = 0.5 * tf.reduce_sum(tf.square(self.mu_z) + tf.square(self.sigma_z) - tf.log(1e-8 + tf.square(self.sigma_z)) - 1, 1)
            mse_loss = tf.reduce_sum(tf.square(self.input_X-self.recons_X), 1)          
            self.all_loss =  mse_loss  
            self.loss = tf.reduce_mean(mse_loss + KL_divergence)
            
        with tf.variable_scope('train'):            
            self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
            

    def _fecth_data(self,input_data):        
        if (self.pointer+1) * self.batch_size  >= input_data.shape[0]:
            return_data = input_data[self.pointer*self.batch_size:,:]
            self.pointer = 0
        else:
            return_data =  input_data[ self.pointer*self.batch_size:(self.pointer+1)*self.batch_size,:]
            self.pointer = self.pointer + 1
        return return_data

    def train(self,train_X):
        for index in range(self.train_iter):
            this_X = self._fecth_data(train_X)
            self.sess.run([self.train_op],feed_dict={ self.input_X: this_X })
        self.arrage_recons_loss(train_X)

        
    def arrage_recons_loss(self,input_data):      # reconsturction error threshold 세팅
        all_losses =  self.sess.run(self.all_loss,feed_dict={self.input_X: input_data })
        self.test_loss = np.percentile(all_losses,(1-self.outliers_fraction)*100)         # train을 통해 reconst loss 설정
                

    def test(self,input_data):
        return_label = []
        print("Reconstruction threshold : ", self.test_loss)
        for index in range(input_data.shape[0]):    # test 길이인 8957
            single_X = input_data[index].reshape(1,-1)
            this_loss = self.sess.run(self.loss,feed_dict={ self.input_X: single_X })
            if this_loss < self.test_loss:
                return_label.append(0)        # 정상
            else:
                return_label.append(1)        # 비정상
        return return_label

if __name__ == '__main__':
    scaler = MinMaxScaler()
    train = pd.read_csv('data/wbc_train.csv') 
    test = pd.read_csv('data/wbc_test.csv')                            # csv로 받도록 바꿔주고
    test_label = pd.read_csv('data/wbc_test_label.csv')                # numpy일때는 그냥 넘겨줬지만,

    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    #print(train.shape, train.values.shape, test.shape, test.values.shape, test_label.shape, test_label.values.shape)
    mlp_vae_predict(pd.DataFrame(train).values, pd.DataFrame(test).values, test_label.values)    # csv일때는 .values로 npy처럼 값으로 넘겨줘서 에러 해결

In [3]:
from itertools import permutations

digits=list(range(2))

tasks_to_add = list(permutations(digits, 1))

tasks_to_add

[(0,), (1,)]

In [11]:
itr = 50

if not (itr % 50) and itr > 0:
    print("아님")

아님


In [1]:
import tensorflow as tf
x = tf.constant([[1., 1.], [2., 2.]])
why = tf.reduce_mean(x)  # 1.5

In [2]:
why

<tf.Tensor 'Mean:0' shape=() dtype=float32>